In [9]:
import dai
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [10]:
ed = dai.query("SELECT DISTINCT date FROM cn_stock_bar1d WHERE date >= '2024-01-01'").df()['date'].max().strftime("%Y-%m-%d")
sd = (pd.Timestamp(ed) - timedelta(days=45)).strftime("%Y-%m-%d")

In [11]:
def get_factor_df_one_instrument_one_day(instrument, date):

    sql = f"""
    WITH 
    data1 AS (
        SELECT 
            instrument,
            date,
            make_date(year(date), month(date),  day(date))    AS day,
            make_time(hour(date), minute(date), second(date)) AS _time,
            LAG(volume, 1) OVER (PARTITION BY day, instrument ORDER BY date) AS _volume_1,
            LAG(volume, 2) OVER (PARTITION BY day, instrument ORDER BY date) AS _volume_2,
            LAG(volume, 3) OVER (PARTITION BY day, instrument ORDER BY date) AS _volume_3,
            LAG(volume, 4) OVER (PARTITION BY day, instrument ORDER BY date) AS _volume_4,
            LAG(volume, 5) OVER (PARTITION BY day, instrument ORDER BY date) AS _volume_5,
            CORR(volume, _volume_1) OVER (PARTITION BY day, instrument) AS ACMA1,
            CORR(volume, _volume_2) OVER (PARTITION BY day, instrument) AS ACMA2,
            CORR(volume, _volume_3) OVER (PARTITION BY day, instrument) AS ACMA3,
            CORR(volume, _volume_4) OVER (PARTITION BY day, instrument) AS ACMA4,
            CORR(volume, _volume_5) OVER (PARTITION BY day, instrument) AS ACMA5,
            IF(_time = '10:09:00',  1, 0) AS _is39,
            IF(_time = '10:10:00',  1, 0) AS _is40,
            IF(_time = '10:11:00',  1, 0) AS _is41,
            IF(_time >= '14:43:00', 1, 0) AS _isL18,
            IF(_time >= '14:42:00', 1, 0) AS _isL19,
            IF(_time >= '14:41:00', 1, 0) AS _isL20,
            IF(_time >= '14:40:00', 1, 0) AS _isL21,
            IF(_time >= '14:39:00', 1, 0) AS _isL22,
            SUM(volume * _is39) OVER(PARTITION BY day, instrument) AS V39,
            SUM(volume * _is40) OVER(PARTITION BY day, instrument) AS V40,
            SUM(volume * _is41) OVER(PARTITION BY day, instrument) AS V41,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume * _isL18) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS APL18,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume * _isL19) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS APL19,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume * _isL20) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS APL20,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume * _isL21) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS APL21,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume * _isL22) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS APL22,
        FROM cn_stock_bar1m
        WHERE date >= '{date} 00:00:00.000' AND date <= '{date} 23:59:59.999'
        AND instrument = '{instrument}'
        ORDER BY date, instrument
    ),

    data2 AS (
        SELECT DISTINCT 
            day AS date,
            instrument,
            V39     AS alpha_hf_00030001,
            V40     AS alpha_hf_00030002,
            V41     AS alpha_hf_00030003,
            APL18   AS alpha_hf_00030004,
            APL19   AS alpha_hf_00030005,
            APL20   AS alpha_hf_00030006,
            APL21   AS alpha_hf_00030007,
            APL22   AS alpha_hf_00030008,
            ACMA1   AS alpha_hf_00030009,
            ACMA2   AS alpha_hf_00030010,
            ACMA3   AS alpha_hf_00030011,
            ACMA4   AS alpha_hf_00030012,
            ACMA5   AS alpha_hf_00030013,
        FROM data1
    )
    SELECT * 
    FROM data2

    """
    data = dai.query(sql).df()
    return data

In [12]:
get_factor_df_one_instrument_one_day('600519.SH', ed)

,date,instrument,alpha_hf_00030001,alpha_hf_00030002,alpha_hf_00030003,alpha_hf_00030004,alpha_hf_00030005,alpha_hf_00030006,alpha_hf_00030007,alpha_hf_00030008,alpha_hf_00030009,alpha_hf_00030010,alpha_hf_00030011,alpha_hf_00030012,alpha_hf_00030013
0,2026-02-13,600519.SH,31300,11200,15500,0.081813,0.086947,0.092658,0.102327,0.107821,0.555811,0.487451,0.390961,0.355209,0.290889
